In [ ]:
import pandas as pd
import numpy as np
import re
import string
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import ISRIStemmer #arabic stemming
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
#!pip  install -U farasapy #limitization arabic $ remove il tashkeel
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from sklearn.model_selection import train_test_split

from sklearn.model_selection import train_test_split
!pip install emoji
import emoji

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 6.9 MB/s eta 0:00:00


In [ ]:

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('arabic')
print(stopwords )


['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:

def preprocessing(data):

    text_no_emoji = emoji.demojize(data)#remove emojies
    data = re.sub(r':[a-zA-Z_]+:', ' ', text_no_emoji)


    review = re.sub("[ًٌٍَُِّْـ]", "", data)#remove tashkeel
    review = re.sub('[A-Za-z0-9]',' ',review)#remove english and numbers

    ##############################################

    # Normalize elongated words
    review = re.sub(r'(\w)\1{2,}', r'\1', review) #اوووو-> او

     # Handling Negations
    review = re.sub(r"\b(?:لا|لم|لن|ما|من|ألا|لست)\b", "not", review)


    ##############################################

    #replace every confused letter with general one
    review = re.sub("[إأآا]", "ا", review)
    review = re.sub("ى", "ي", review)
    review = re.sub("ؤ", "ء",review)
    review = re.sub("ئ", "ء",review)
    review = re.sub("ة", "ه", review)
    review = re.sub("گ", "ك", review)

    punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation #remove punctuations
    translator = str.maketrans('', '', punctuations)
    review = review.translate(translator)


    #replace spaces and tabs and so on
    review = re.sub(r"\n+", ' ', review)
    review = re.sub(r"\t+", ' ', review)
    review= re.sub(r"\r+", ' ',review)
    review = re.sub(r"\s+", ' ', review)

    review=nltk.word_tokenize(review)#tokenization

    review=[w for w in review if not w in stopwords ]#remove stop word
    isri_stemmer = ISRIStemmer() #stemming for arabic
    review=[ISRIStemmer().suf32(w)  for w in review]
    review = ' '.join(review)
    return review




In [ ]:
data = pd.read_excel('/content/train.xlsx')
xtrain= data.review_description.apply(preprocessing)#applies the preprocessing function to each element in the 'review_description' column of the DataFrame
ytrain=data['rating']

In [ ]:
print(xtrain[0])

شركه زباله سواق بتبرشم مفيش حتي رقم للشكاوي السواق يسيبك يمشي ميرضاش يقف يقولك اعملك ايه اتصل بالشركه قول مرضيش يقفلي رغم اني حاجز not قبل بيوم


In [ ]:
#remove spaces and drop every empty row
j=0

for text in xtrain:
  if text.strip():
    xtrain[j]=text.strip()
  else :

    xtrain = xtrain.drop(j)
    ytrain=ytrain.drop(j)
  j+=1


#for text, label in zip(xtrain, ytrain):
    #if text.strip():  # Check if the text is not an empty string after stripping whitespace
        #X_train.append(text)
       # Y_train.append(label)
#print(len(c),len(a))
#print(c)
#print(len( X_train),len(Y_train))

print(len(xtrain),len(ytrain))

30374 30374


In [ ]:
  tokenizer = Tokenizer() #toknize and padd
  tokenizer.fit_on_texts(xtrain)
  vocab_size = len(tokenizer.word_index) + 1
  sequences = tokenizer.texts_to_sequences(xtrain)
  max_length = max(len(seq) for seq in sequences)
  X_train = pad_sequences(sequences, maxlen=max_length, padding='post')

In [ ]:
# Convert labels to one-hot encoding

y_train = np.zeros((len(ytrain),3))
for i, label in enumerate(ytrain):
    if label == 1:
        y_train[i, 0] = 1  # Positive
    elif label == -1:
        y_train[i, 1] = 1  # Negative
    else:
        y_train[i, 2] = 1  # Neutral

#print(y_train)


In [ ]:
print(vocab_size,max_length)

31024 327


In [ ]:
print( y_train[0],y_train[6],y_train[5])

[0. 1. 0.] [1. 0. 0.] [0. 0. 1.]


In [ ]:
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train,test_size=0.2, random_state=42)

In [ ]:
#bidirectional lstm

from tensorflow.keras.callbacks import EarlyStopping
from keras.utils import to_categorical

from keras.utils import to_categorical


model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))
model.add(Bidirectional(LSTM(100)))
#model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_final, y_train_final, epochs=2, batch_size=32, validation_data=(X_val, y_val))

loss, accuracy = model.evaluate(X_train_final, y_train_final, verbose=True)
print("Training Accuracy: {:.4f}".format(accuracy))
loss_val, accuracy_val = model.evaluate(X_val, y_val, verbose=True)
print("Testing Accuracy:  {:.4f}".format(accuracy_val))



Epoch 1/2
760/760 [==============================] - 540s 699ms/step - loss: 0.5051 - accuracy: 0.8086 - val_loss: 0.4731 - val_accuracy: 0.8272
Epoch 2/2
760/760 [==============================] - 133s 174ms/step - loss: 0.2351 - accuracy: 0.9173
Training Accuracy: 0.9173
190/190 [==============================] - 34s 180ms/step - loss: 0.5123 - accuracy: 0.8056
Testing Accuracy:  0.8056


In [ ]:
#Simple DNN Model feed forward
from tensorflow.keras.layers import Embedding, Flatten, Dense

model2 = Sequential()
model2.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))
model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dense(3, activation='softmax'))

# Compile the model
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.fit(X_train_final, y_train_final, epochs=2, batch_size=32, validation_data=(X_val, y_val))

loss, accuracy = model2.evaluate(X_train_final, y_train_final, verbose=True)
print("Training Accuracy: {:.4f}".format(accuracy))
loss_val, accuracy_val = model2.evaluate(X_val, y_val, verbose=True)
print("Testing Accuracy:  {:.4f}".format(accuracy_val))


Epoch 1/2
760/760 [==============================] - 200s 263ms/step - loss: 0.5048 - accuracy: 0.8105 - val_loss: 0.4795 - val_accuracy: 0.8250
Epoch 2/2
760/760 [==============================] - 17s 22ms/step - loss: 0.1192 - accuracy: 0.9681
Training Accuracy: 0.9681
190/190 [==============================] - 4s 23ms/step - loss: 0.5596 - accuracy: 0.7972
Testing Accuracy:  0.7972


In [ ]:
# Model CNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
modelCNN = Sequential()
modelCNN.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_length))
modelCNN.add(GlobalMaxPooling1D())
modelCNN.add(Dense(256, activation='relu'))
modelCNN.add(Dropout(0.5))  # Adding dropout layer with 50% dropout rate
modelCNN.add(Dense(3, activation='softmax'))

# Compile the model
modelCNN.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelCNN.fit(X_train_final, y_train_final, epochs=2, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = modelCNN.evaluate(X_val, y_val, verbose=True)
print(f'Validation accuracy: {accuracy * 100:.2f}%')

Epoch 1/2
760/760 [==============================] - 32s 41ms/step - loss: 0.5344 - accuracy: 0.7938 - val_loss: 0.4896 - val_accuracy: 0.8171
Epoch 2/2
190/190 [==============================] - 0s 2ms/step - loss: 0.5043 - accuracy: 0.8133
Validation accuracy: 81.33%


In [ ]:
from keras import layers
#simple rnn
model_rnn_arabic = Sequential()
model_rnn_arabic.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length))
model_rnn_arabic.add(layers.SimpleRNN(8, return_sequences=True, activation='tanh'))
model_rnn_arabic.add(layers.SimpleRNN(4, activation='tanh'))
model_rnn_arabic.add(Dense(units=3, activation='softmax'))

model_rnn_arabic.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Train the model
model_rnn_arabic.fit(X_train_final, y_train_final, epochs=4, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the validation set
loss_rnn_arabic, accuracy_rnn_arabic = model_rnn_arabic.evaluate(X_val, y_val, verbose=True)
print("Validation Accuracy: {:.4f}".format(accuracy_rnn_arabic))


In [ ]:
#feed forward neural network for text calssification cutomized
from tensorflow.keras.callbacks import EarlyStopping

model3 = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='tanh'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 output classes: positive, negative, neutral
])

model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True)

# Train the model
num_epochs = 10
model3.fit(X_train_final, y_train_final, epochs=num_epochs, validation_data=(X_val, y_val),callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model3.evaluate(X_val, y_val, verbose=True)
print(f'Validation accuracy: {accuracy * 100:.2f}%')


Epoch 1/10
760/760 [==============================] - 13s 16ms/step - loss: 0.7557 - accuracy: 0.6496 - val_loss: 0.5961 - val_accuracy: 0.7807
Epoch 2/10
760/760 [==============================] - 10s 13ms/step - loss: 0.4994 - accuracy: 0.8226 - val_loss: 0.5581 - val_accuracy: 0.7886
Epoch 3/10
760/760 [==============================] - 10s 13ms/step - loss: 0.4245 - accuracy: 0.8552 - val_loss: 0.5311 - val_accuracy: 0.8076
Epoch 4/10
760/760 [==============================] - 10s 13ms/step - loss: 0.3808 - accuracy: 0.8725 - val_loss: 0.5332 - val_accuracy: 0.8142
Epoch 5/10
760/760 [==============================] - 10s 13ms/step - loss: 0.3454 - accuracy: 0.8859 - val_loss: 0.5636 - val_accuracy: 0.8067
Epoch 6/10
760/760 [==============================] - 10s 14ms/step - loss: 0.3088 - accuracy: 0.8964 - val_loss: 0.5589 - val_accuracy: 0.8170
Epoch 7/10
760/760 [==============================] - 10s 14ms/step - loss: 0.2847 - accuracy: 0.9059 - val_loss: 0.6131 - val_accuracy:

In [ ]:
#another cnn
modeltest = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 32, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 output classes: positive, negative, neutral
])

modeltest.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
num_epochs = 10
modeltest.fit(X_train_final, y_train_final, epochs=num_epochs, validation_data=(X_val, y_val))

# Evaluate the model
loss, accuracy = modeltest.evaluate(X_val, y_val, verbose=True)
print(f'Validation accuracy: {accuracy * 100:.2f}%')


Epoch 1/10
760/760 [==============================] - 9s 11ms/step - loss: 0.8313 - accuracy: 0.5876 - val_loss: 0.8144 - val_accuracy: 0.5916
Epoch 2/10
760/760 [==============================] - 7s 9ms/step - loss: 0.7347 - accuracy: 0.6634 - val_loss: 0.6721 - val_accuracy: 0.7164
Epoch 3/10
760/760 [==============================] - 6s 8ms/step - loss: 0.5859 - accuracy: 0.7732 - val_loss: 0.5874 - val_accuracy: 0.7913
Epoch 4/10
760/760 [==============================] - 5s 7ms/step - loss: 0.5000 - accuracy: 0.8222 - val_loss: 0.5402 - val_accuracy: 0.8000
Epoch 5/10
760/760 [==============================] - 5s 7ms/step - loss: 0.4518 - accuracy: 0.8451 - val_loss: 0.5249 - val_accuracy: 0.8036
Epoch 6/10
760/760 [==============================] - 5s 7ms/step - loss: 0.4226 - accuracy: 0.8575 - val_loss: 0.5150 - val_accuracy: 0.8216
Epoch 7/10
760/760 [==============================] - 5s 7ms/step - loss: 0.3980 - accuracy: 0.8674 - val_loss: 0.5102 - val_accuracy: 0.8201
Epoch

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.5):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class PositionalEncoding(layers.Layer):
    def __init__(self, max_len, d_model):
        super(PositionalEncoding, self).__init__()
        self.position = tf.range(max_len, dtype=tf.float32)[:, tf.newaxis]
        self.d_model = d_model

    def call(self, inputs):
        i = tf.range(0, self.d_model, 2, dtype=tf.float32)
        den = 1 / tf.pow(tf.cast(10000, tf.float32), i / tf.cast(self.d_model, tf.float32))
        sines = tf.math.sin(self.position * den)
        cosines = tf.math.cos(self.position * den)

        pos_encoding = tf.concat([sines, cosines], axis=-1)

        return tf.cast(pos_encoding+inputs, dtype=tf.float32)

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = PositionalEncoding(maxlen, embed_dim)
        self.max_len = maxlen

    def call(self, x):
        x = self.token_emb(x)
        positions = self.pos_emb(x)

        return tf.cast(positions, dtype=tf.float32)

x_train = keras.utils.pad_sequences(X_train_final, maxlen=max_length)
x_val = keras.utils.pad_sequences(X_val, maxlen=max_length)

# print(x_train)
# print(x_val)
embed_dim = 32  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(max_length,))
embedding_layer = TokenAndPositionEmbedding(max_length, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(15, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(3, activation="softmax")(x)

model3 = keras.Model(inputs=inputs, outputs=outputs)

model3.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
history = model3.fit(
    x_train, y_train_final, epochs=5, validation_data=(X_val, y_val)
)
model3.summary()

# loss, accuracy = model3.evaluate(X_train_final, y_train_final, verbose=True)
# print("Training Accuracy: {:.4f}".format(accuracy))
loss_val, accuracy_val = model3.evaluate(X_val, y_val, verbose=True)
print("Testing Accuracy:  {:.4f}".format(accuracy_val))

Epoch 1/5
760/760 [==============================] - 485s 634ms/step - loss: 0.8729 - accuracy: 0.5728 - val_loss: 0.8398 - val_accuracy: 0.5868
Epoch 2/5
760/760 [==============================] - 480s 632ms/step - loss: 0.6946 - accuracy: 0.6957 - val_loss: 0.5342 - val_accuracy: 0.8038
Epoch 3/5
760/760 [==============================] - 479s 629ms/step - loss: 0.4576 - accuracy: 0.8393 - val_loss: 0.5414 - val_accuracy: 0.8148
Epoch 4/5
760/760 [==============================] - 481s 633ms/step - loss: 0.3709 - accuracy: 0.8775 - val_loss: 0.5879 - val_accuracy: 0.8054
Epoch 5/5
760/760 [==============================] - 489s 644ms/step - loss: 0.3171 - accuracy: 0.8979 - val_loss: 0.6313 - val_accuracy: 0.8086
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 327)]             0         
                                                            

In [ ]:
testing_data= pd.read_csv("/content/test _no_label.csv")
X_test= testing_data.review_description.apply(preprocessing)#applies the clean_text function to each element in the 'review_description' column of the DataFrame

print(X_test)

0      اهنء علي خدمه العملاء المحادثه المباشره not قص...
1      ممتاز جدا اتم ان تكون المسابق والجواءز طلب الس...
2              محملته يقول تم ايقاف حطيت عشان تسو الخطاء
3                                                شغل طيب
4                                                 ماجربت
                             ...                        
995                                                يستهل
996                               خدمه سيءه بكل المعايير
997                                        لءي٠٣٣٢لء٣٤٣س
998                               تطبيق صادق خصم الكوبون
999    تاخير الموظف مءهله للاسف انا استخدم طلب not سن...
Name: review_description, Length: 1000, dtype: object


In [ ]:
j=0
for text in X_test:
    X_test[j]=text.strip()
    j=j+1

In [ ]:
model_input=tokenizer.texts_to_sequences(X_test)
model_input=pad_sequences(model_input, padding='post', maxlen=max_length)

In [ ]:
pred = modeltest.predict(model_input, verbose=True)


In [ ]:
predicted_labels = np.argmax(pred, axis=1)

In [ ]:
predicted_labels

In [ ]:
import csv
sentiment_labels = [1,-1,0] #positive negative neutral
id=1
with open('predictions.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ID', 'rating'])
    for  label in predicted_labels:
        sentiment = sentiment_labels[label]
        writer.writerow([id, sentiment])
        id+=1

print("Predictions saved to predictions.csv")

Predictions saved to predictions.csv
